In [16]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:86% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:130px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:15px;}
</style>
"""))

In [17]:
# 필요 라이브러리
import pymysql
import pandas as pd
from dotenv import load_dotenv
import os

df = pd.read_csv(r"C:\Users\wjdwh\workspace\project\--PM----\DL_models\LSTM최종데이터_X_Y_라벨링등전.csv")
df

,연도,월,일,시,행정구,일시,대여량,총생활인구수,계절,출퇴근시간,주말구분,강수,기온,습도,풍속
0,2021,1,1,0,강남구,2021-01-01 00:00:00,9.0,618455.4297,겨울,기타,0,0.0,-4.2,59.0,1.4
1,2021,1,1,0,강동구,2021-01-01 00:00:00,12.0,525220.6638,겨울,기타,0,0.0,-6.6,60.0,1.4
2,2021,1,1,0,강북구,2021-01-01 00:00:00,12.0,321499.2460,겨울,기타,0,0.0,-5.4,54.0,1.1
3,2021,1,1,0,강서구,2021-01-01 00:00:00,27.0,547287.6699,겨울,기타,0,0.0,-3.3,52.0,0.9
4,2021,1,1,0,관악구,2021-01-01 00:00:00,19.0,492742.3420,겨울,기타,0,0.0,-2.5,50.0,0.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
962638,2025,5,31,23,용산구,2025-05-31 23:00:00,NaN,258570.4031,봄,기타,1,0.0,21.7,62.0,1.2
962639,2025,5,31,23,은평구,2025-05-31 23:00:00,NaN,450100.7857,봄,기타,1,0.0,22.4,62.0,0.9
962640,2025,5,31,23,종로구,2025-05-31 23:00:00,NaN,207295.4128,봄,기타,1,0.0,22.0,66.0,2.1
962641,2025,5,31,23,중구,2025-05-31 23:00:00,NaN,156366.4837,봄,기타,1,0.0,22.0,66.0,2.1


In [26]:
load_dotenv()
host = os.getenv('HOST')
user = os.getenv('USER')
password = os.getenv('PASSWORD')
database = os.getenv('DATABASE')

In [45]:
# MySQL 연결 정보
connection = pymysql.connect(host=host,
                                       user=user,
                                       password=password,
                                       database=database,
                                       charset='utf8mb4',
                                       cursorclass=pymysql.cursors.DictCursor)

try:
    with connection.cursor() as cursor:
        insert_query = """INSERT INTO basic_data (year, month, day, hour, district, datetime, total_population,season, commute_time, weekend_flag, precipitation,temperature, humidity, wind_speed, rental_count) VALUES (%s, %s, %s, %s, %s, %s, %s,%s, %s, %s, %s,%s, %s, %s, %s)"""
        for _, row in df.iterrows():
            dt = f"{row['연도']}-{row['월']:02d}-{row['일']:02d} {row['시']:02d}:00:00"
            rental_count = None if pd.isna(row['대여량']) else int(row['대여량'])
            cursor.execute(insert_query, (row['연도'], row['월'], row['일'], row['시'], row['행정구'], dt, row['총생활인구수'],row['계절'], row['출퇴근시간'], bool(row['주말구분']), row['강수'],row['기온'], row['습도'], row['풍속'], rental_count))
    connection.commit()
    
    print("데이터 삽입 완료!")

finally:
    connection.close()


데이터 삽입 완료!


In [4]:
import pymysql
import pandas as pd
from dotenv import load_dotenv
import os

load_dotenv()
host = os.getenv('host')
user = os.getenv('user')
password = os.getenv('password')
database = os.getenv('database')
port = os.getenv('port')

connection = pymysql.connect(host=host,
                                       user=user,
                                       password=password,
                                       database=database,
                                       charset='utf8mb4',
                                       cursorclass=pymysql.cursors.DictCursor)

try:
    with connection.cursor() as cursor:
        query = "SELECT * FROM basic_data"
        cursor.execute(query)
        data = cursor.fetchall()
        print(pd.DataFrame(data))
except :
    print('에러 발생')

finally:
    connection.close()

        year  month  day  hour district            datetime  total_population  \
0       2021      1    1     0      강남구 2021-01-01 00:00:00            618455   
1       2021      1    1     0      강동구 2021-01-01 00:00:00            525221   
2       2021      1    1     0      강북구 2021-01-01 00:00:00            321499   
3       2021      1    1     0      강서구 2021-01-01 00:00:00            547288   
4       2021      1    1     0      관악구 2021-01-01 00:00:00            492742   
...      ...    ...  ...   ...      ...                 ...               ...   
962638  2025      5   31    23      용산구 2025-05-31 23:00:00            258570   
962639  2025      5   31    23      은평구 2025-05-31 23:00:00            450101   
962640  2025      5   31    23      종로구 2025-05-31 23:00:00            207295   
962641  2025      5   31    23       중구 2025-05-31 23:00:00            156366   
962642  2025      5   31    23      중랑구 2025-05-31 23:00:00            348621   

       season commute_time 